In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2019-06-19 09:00:16--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  5.34MB/s    in 0.9s    

2019-06-19 09:00:17 (5.34 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflat

In [2]:
%cd ml-100k
!shuf ua.base -o ua.base.shuffled
#User id, Movie id, Movieratings, Timestamp
!head -10 ua.base.shuffled

/home/ec2-user/SageMaker/ml-100k
454	327	3	881958428
878	530	5	880872619
62	232	3	879375977
398	95	5	875659266
747	1194	5	888639102
297	282	3	874954845
627	1135	3	879530625
796	945	5	892663009
15	455	1	879455914
916	1113	4	880844897


In [3]:
!head -10 ua.test

1	20	4	887431883
1	33	4	878542699
1	61	4	878542420
1	117	3	874965739
1	155	2	878542201
1	160	4	875072547
1	171	5	889751711
1	189	3	888732928
1	202	5	875072442
1	265	4	878542441


In [4]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer

import boto3, csv, io, json
import numpy as np
from scipy.sparse import lil_matrix

In [5]:
nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies

nbRatingsTrain=90570
nbRatingsTest=9430

In [6]:
# For each user, build a list of rated movies.
# We'd need this to add random negative samples.
moviesByUser = {}
for userId in range(nbUsers):
    moviesByUser[str(userId)]=[]
 
with open('ua.base.shuffled','r') as f:
    samples=csv.reader(f,delimiter='\t')
    for userId,movieId,rating,timestamp in samples:
        moviesByUser[str(int(userId)-1)].append(int(movieId)-1)

In [7]:
def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            if int(rating) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1
            
    Y=np.array(Y).astype('float32')
    return X,Y

In [8]:
X_train, Y_train = loadDataset('ua.base.shuffled', nbRatingsTrain, nbFeatures)
X_test, Y_test = loadDataset('ua.test',nbRatingsTest,nbFeatures)

In [9]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nbRatingsTrain, nbFeatures)
assert Y_train.shape == (nbRatingsTrain, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nbRatingsTrain-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nbRatingsTest, nbFeatures)
assert Y_test.shape  == (nbRatingsTest, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nbRatingsTest-zero_labels))

(90570, 2625)
(90570,)
Training labels: 49906 zeros, 40664 ones
(9430, 2625)
(9430,)
Test labels: 5469 zeros, 3961 ones


In [10]:
# Convert to protobuf and save to s3
bucket = 'ml-trainings'
prefix = 'sagemaker/fm-movielens'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train3')
# Convert to protobuf and save to s3train_prefix   = '{}/{}'.format(prefix, 'train3')


test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test3')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [11]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

s3://ml-trainings/sagemaker/fm-movielens/train3/train.protobuf
s3://ml-trainings/sagemaker/fm-movielens/test3/test.protobuf
Output: s3://ml-trainings/sagemaker/fm-movielens/output


In [12]:
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest'}

In [13]:
fm = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': train_data, 'test': test_data})

2019-06-19 09:01:21 Starting - Starting the training job...
2019-06-19 09:01:23 Starting - Launching requested ML instances......
2019-06-19 09:02:27 Starting - Preparing the instances for training...
2019-06-19 09:03:10 Downloading - Downloading input data.....
Docker entrypoint called with argument(s): train
[06/19/2019 09:04:00 INFO 140517020325696] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'bias_init_method': u'normal', u'linear_init_method': u'normal', u'linear_lr': u'0.001', u'factors_init_method': u'normal', u'_tuning_objective_metric': u'', u'bias_wd': u'0.01', u'use_linear': u'true', u'bias_lr': u'0.1', u'mini_batch_size': u'1000', u'_use_full_symbolic': u'true', u'batch_metrics_publish_interval': u'500', u'bias_


2019-06-19 09:04:00 Training - Training image download completed. Training in progress.[2019-06-19 09:04:10.668] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 32, "duration": 607, "num_examples": 91, "num_bytes": 5796480}
[06/19/2019 09:04:10 INFO 140517020325696] #quality_metric: host=algo-1, epoch=15, train binary_classification_accuracy <score>=0.723362637363
[06/19/2019 09:04:10 INFO 140517020325696] #quality_metric: host=algo-1, epoch=15, train binary_classification_cross_entropy <loss>=0.589194858425
[06/19/2019 09:04:10 INFO 140517020325696] #quality_metric: host=algo-1, epoch=15, train binary_f_1.000 <score>=0.767891718453
#metrics {"Metrics": {"update.time": {"count": 1, "max": 609.6141338348389, "sum": 609.6141338348389, "min": 609.6141338348389}}, "EndTime": 1560935050.668955, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1560935050.058442}

[06/19/2019 09:04:10 INFO 1405

[2019-06-19 09:04:20.787] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 66, "duration": 613, "num_examples": 91, "num_bytes": 5796480}
[06/19/2019 09:04:20 INFO 140517020325696] #quality_metric: host=algo-1, epoch=32, train binary_classification_accuracy <score>=0.733637362637
[06/19/2019 09:04:20 INFO 140517020325696] #quality_metric: host=algo-1, epoch=32, train binary_classification_cross_entropy <loss>=0.5545506404
[06/19/2019 09:04:20 INFO 140517020325696] #quality_metric: host=algo-1, epoch=32, train binary_f_1.000 <score>=0.769211726508
#metrics {"Metrics": {"update.time": {"count": 1, "max": 615.2670383453369, "sum": 615.2670383453369, "min": 615.2670383453369}}, "EndTime": 1560935060.788021, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1560935060.171904}

[06/19/2019 09:04:20 INFO 140517020325696] #progress_metric: host=algo-1, completed 33 % of epochs
#metrics {"Metrics": 

[2019-06-19 09:04:30.815] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 100, "duration": 582, "num_examples": 91, "num_bytes": 5796480}
[06/19/2019 09:04:30 INFO 140517020325696] #quality_metric: host=algo-1, epoch=49, train binary_classification_accuracy <score>=0.742241758242
[06/19/2019 09:04:30 INFO 140517020325696] #quality_metric: host=algo-1, epoch=49, train binary_classification_cross_entropy <loss>=0.537760753254
[06/19/2019 09:04:30 INFO 140517020325696] #quality_metric: host=algo-1, epoch=49, train binary_f_1.000 <score>=0.773804702116
#metrics {"Metrics": {"update.time": {"count": 1, "max": 584.2840671539307, "sum": 584.2840671539307, "min": 584.2840671539307}}, "EndTime": 1560935070.816264, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1560935070.231143}

[06/19/2019 09:04:30 INFO 140517020325696] #progress_metric: host=algo-1, completed 50 % of epochs
#metrics {"Metrics

[2019-06-19 09:04:40.857] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 134, "duration": 573, "num_examples": 91, "num_bytes": 5796480}
[06/19/2019 09:04:40 INFO 140517020325696] #quality_metric: host=algo-1, epoch=66, train binary_classification_accuracy <score>=0.745274725275
[06/19/2019 09:04:40 INFO 140517020325696] #quality_metric: host=algo-1, epoch=66, train binary_classification_cross_entropy <loss>=0.527191444145
[06/19/2019 09:04:40 INFO 140517020325696] #quality_metric: host=algo-1, epoch=66, train binary_f_1.000 <score>=0.775908739366
#metrics {"Metrics": {"update.time": {"count": 1, "max": 575.5369663238525, "sum": 575.5369663238525, "min": 575.5369663238525}}, "EndTime": 1560935080.857946, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1560935080.281548}

[06/19/2019 09:04:40 INFO 140517020325696] #progress_metric: host=algo-1, completed 67 % of epochs
#metrics {"Metrics

[2019-06-19 09:04:46.234] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 152, "duration": 622, "num_examples": 91, "num_bytes": 5796480}
[06/19/2019 09:04:46 INFO 140517020325696] #quality_metric: host=algo-1, epoch=75, train binary_classification_accuracy <score>=0.746538461538
[06/19/2019 09:04:46 INFO 140517020325696] #quality_metric: host=algo-1, epoch=75, train binary_classification_cross_entropy <loss>=0.52297908674
[06/19/2019 09:04:46 INFO 140517020325696] #quality_metric: host=algo-1, epoch=75, train binary_f_1.000 <score>=0.776932078647
#metrics {"Metrics": {"update.time": {"count": 1, "max": 624.0320205688477, "sum": 624.0320205688477, "min": 624.0320205688477}}, "EndTime": 1560935086.234839, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1560935085.609906}

[06/19/2019 09:04:46 INFO 140517020325696] #progress_metric: host=algo-1, completed 76 % of epochs
#metrics {"Metrics"


2019-06-19 09:05:10 Uploading - Uploading generated training model
2019-06-19 09:05:10 Completed - Training job completed
Billable seconds: 120


In [ ]:
fm_predictor = fm.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)

----------------------------------------------------------------------------

In [ ]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    #print js
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

In [21]:
result = fm_predictor.predict(X_test[1000:1010].toarray())
print(result)
print (Y_test[1000:1010])

{'predictions': [{'score': 0.6684710383415222, 'predicted_label': 1.0}, {'score': 0.1921505481004715, 'predicted_label': 0.0}, {'score': 0.23042139410972595, 'predicted_label': 0.0}, {'score': 0.6075059771537781, 'predicted_label': 1.0}, {'score': 0.530755877494812, 'predicted_label': 1.0}, {'score': 0.14946553111076355, 'predicted_label': 0.0}, {'score': 0.39576730132102966, 'predicted_label': 0.0}, {'score': 0.518810510635376, 'predicted_label': 1.0}, {'score': 0.35530826449394226, 'predicted_label': 0.0}, {'score': 0.11803825944662094, 'predicted_label': 0.0}]}
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
